In [1]:
%load_ext autoreload
%autoreload 3

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
import networkx as nx
import scipy
import sklearn
import time
import pickle
import warnings

# methods
from gaccord import GraphicalAccord, GraphicalConcord
from inverse_covariance import QuicGraphicalLasso
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri

# utils
import sys
sys.path.append('../utils')
from utils import standardize, partial_corr, tp_fp, mcc, precision_recall, pseudo_BIC, gauss_BIC, proj_precision_mat, newton_one_step
from generate_graphs import generate_erdos_renyi, generate_data

In [2]:
def compute_norm(Theta, Theta_hat):
    p = len(Theta)
    Theta_bool = np.where(Theta != 0, 1, 0)
    Theta_hat_bool = np.where(Theta_hat != 0, 1, 0)

    mask = np.tri(p, p, k=-1, dtype=bool)
    edge_true = Theta_bool[mask]
    edge_hat = Theta_hat_bool[mask]
    TP_indx = np.where((edge_true == 1) & (edge_hat == 1) == 1)[0]
    FP_indx = np.where((edge_true == 0) & (edge_hat == 1) == 1)[0]

    theta_true = Theta[mask]
    theta_hat = Theta_hat[mask]

    norm_TP = np.linalg.norm(theta_true[TP_indx] - theta_hat[TP_indx])
    norm_FP = np.linalg.norm(theta_true[FP_indx] - theta_hat[FP_indx])
    norm_total = np.linalg.norm(theta_true - theta_hat)
    norm_TP_average = np.linalg.norm(theta_true[TP_indx] - theta_hat[TP_indx])/len(TP_indx)
    norm_FP_average = np.linalg.norm(theta_true[FP_indx] - theta_hat[FP_indx])/len(FP_indx)

    return norm_TP, norm_FP, norm_total, norm_TP_average, norm_FP_average

In [3]:
# select graph structure = ['hub_network', 'erdos_renyi']
graph_structure = 'hub_network'

### Generate graph and data

In [4]:
if graph_structure == 'hub_network':
    # we use pre-made hub-network graph structure, which was constructed by the following procedure:
    # (1) create a Barabasi-Albert scale-free graph
    # (2) randomly choose 5% of the nodes to be hub nodes
    # (3) for each hub node, construct a complete sub-graph (clique)
    Skel = np.genfromtxt('../data/hub_network_structure.txt', delimiter=',')

    n, p = 500, 1000
    n_prop_to_p = [0.5]
    random_state = 2023
    lower_weight, upper_weight = 0.5, 1.0

    # projection method
    np.random.seed(random_state)
    edge_weights = np.random.uniform(low=lower_weight, high=upper_weight, size=(p,p))
    edge_signs = np.random.choice([-1,1], size=(p,p))
    Theta = np.multiply(edge_weights, edge_signs)
    Theta = np.multiply(Skel, Theta)
    Theta = np.tril(Theta) + np.tril(Theta).T
    nz_indx = np.nonzero(Theta)
    for i in range(100):
        Theta = proj_precision_mat(Theta, nz_indx)
        if np.linalg.cond(Theta) < 20:
            break

    Theta = np.real(Theta)
    # spread diagonal of precision matrix
    spread_diag=[1, 3]
    d = np.random.uniform(spread_diag[0], spread_diag[1], p)
    Theta = np.diag(d) @ Theta @ np.diag(d)
    Rho = partial_corr(Theta)
    Sigma = np.linalg.inv(Theta)

    Xs = generate_data(p, n_prop_to_p, Sigma, N=1, standardize=False, random_state=2023)
    X = Xs[0]

elif graph_structure == 'erdos_renyi':
    n, p = 500, 1000
    n_prop_to_p = [0.5]
    Theta, Sigma = generate_erdos_renyi(p, type='proj', edge_prob=0.01, lower_weight=0.5, upper_weight=1.0, spread_diag=[1, 3], random_state=2023)
    Rho = partial_corr(Theta)

    Xs = generate_data(p, n_prop_to_p, Sigma, N=1, standardize=False, random_state=2023)
    X = Xs[0]

### Run ACCORD

In [5]:
accord_p_bics, accord_ep_bics_02 = [], []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_accord = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_accord = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_accord:
    # ACCORD
    model = GraphicalAccord(Omega_star=np.eye(p), lam1=lam, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Omega_hat = model.omega_.toarray()
    Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
    
    accord_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    accord_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_accord = lams_accord[np.argmin(accord_p_bics)]
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

# TP_norm_accord_pbic, FP_norm_accord_pbic, total_norm_accord_pbic, TP_average_norm_accord_pbic, FP_average_norm_accord_pbic = compute_norm(Theta, Theta_hat)
TP_norm_accord_pbic, FP_norm_accord_pbic, total_norm_accord_pbic, TP_average_norm_accord_pbic, FP_average_norm_accord_pbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_accord_pbic = np.linalg.norm(Theta_hat - Theta)

# use optimal lambda based on ep-bic
best_lam_accord = lams_accord[np.argmin(accord_ep_bics_02)]
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

# TP_norm_accord_epbic, FP_norm_accord_epbic, total_norm_accord_epbic, TP_average_norm_accord_epbic, FP_average_norm_accord_epbic = compute_norm(Theta, Theta_hat)
TP_norm_accord_epbic, FP_norm_accord_epbic, total_norm_accord_epbic, TP_average_norm_accord_epbic, FP_average_norm_accord_epbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_accord_epbic = np.linalg.norm(Theta_hat - Theta)

### Run ACCORD-SP (version 1, average nonzeros)

In [6]:
accord_sp1_p_bics, accord_sp1_ep_bics_02 = [], []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_accord_sp1 = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_accord_sp1 = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_accord_sp1:
    # ACCORD
    model = GraphicalAccord(Omega_star=np.eye(p), lam1=lam, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Omega_hat = model.omega_.toarray()

    # set w_ij = w_ji = 0 if at least one of them is 0 and average nonzero values
    zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
    Omega_hat[zero_indices] = 0
    Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
    
    accord_sp1_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    accord_sp1_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_accord = lams_accord[np.argmin(accord_sp1_p_bics)]
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
# set w_ij = w_ji = 0 if at least one of them is 0 and average nonzero values
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

# TP_norm_accord_sp1_pbic, FP_norm_accord_sp1_pbic, total_norm_accord_sp1_pbic, TP_average_norm_accord_sp1_pbic, FP_average_norm_accord_sp1_pbic = compute_norm(Theta, Theta_hat)
TP_norm_accord_sp1_pbic, FP_norm_accord_sp1_pbic, total_norm_accord_sp1_pbic, TP_average_norm_accord_sp1_pbic, FP_average_norm_accord_sp1_pbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_accord_sp1_pbic = np.linalg.norm(Theta_hat - Theta)

# use optimal lambda based on ep-bic
best_lam_accord = lams_accord[np.argmin(accord_sp1_ep_bics_02)]
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
# set w_ij = w_ji = 0 if at least one of them is 0 and average nonzero values
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

# TP_norm_accord_sp1_epbic, FP_norm_accord_sp1_epbic, total_norm_accord_sp1_epbic, TP_average_norm_accord_sp1_epbic, FP_average_norm_accord_sp1_epbic = compute_norm(Theta, Theta_hat)
TP_norm_accord_sp1_epbic, FP_norm_accord_sp1_epbic, total_norm_accord_sp1_epbic, TP_average_norm_accord_sp1_epbic, FP_average_norm_accord_sp1_epbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_accord_sp1_epbic = np.linalg.norm(Theta_hat - Theta)

### Run ACCORD-SP (version 2, take a smaller magnitude)

In [7]:
accord_sp2_p_bics, accord_sp2_ep_bics_02 = [], []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_accord_sp2 = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_accord_sp2 = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_accord_sp2:
    # ACCORD
    model = GraphicalAccord(Omega_star=np.eye(p), lam1=lam, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Omega_hat = model.omega_.toarray()

    # set w_ij = w_ji = 0 if at least one of them is 0 and choose the smaller nonzero otherwise
    zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
    Omega_hat[zero_indices] = 0
    nonzero_indices = np.where((Omega_hat != 0) & (Omega_hat.T != 0))
    for i, j in zip(*nonzero_indices):
        if abs(Omega_hat[i, j]) > abs(Omega_hat[j, i]):
            Omega_hat[i, j] = Omega_hat[j, i]
        else:
            Omega_hat[j, i] = Omega_hat[i, j]
    Theta_hat = Omega_hat
    
    accord_sp2_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    accord_sp2_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_accord = lams_accord[np.argmin(accord_sp2_p_bics)]
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
# set w_ij = w_ji = 0 if at least one of them is 0 and choose the smaller nonzero otherwise
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
nonzero_indices = np.where((Omega_hat != 0) & (Omega_hat.T != 0))
for i, j in zip(*nonzero_indices):
    if abs(Omega_hat[i, j]) > abs(Omega_hat[j, i]):
        Omega_hat[i, j] = Omega_hat[j, i]
    else:
        Omega_hat[j, i] = Omega_hat[i, j]
Theta_hat = Omega_hat
Rho_hat = partial_corr(Theta_hat)

# TP_norm_accord_sp2_pbic, FP_norm_accord_sp2_pbic, total_norm_accord_sp2_pbic, TP_average_norm_accord_sp2_pbic, FP_average_norm_accord_sp2_pbic = compute_norm(Theta, Theta_hat)
TP_norm_accord_sp2_pbic, FP_norm_accord_sp2_pbic, total_norm_accord_sp2_pbic, TP_average_norm_accord_sp2_pbic, FP_average_norm_accord_sp2_pbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_accord_sp2_pbic = np.linalg.norm(Theta_hat - Theta)

# use optimal lambda based on ep-bic
best_lam_accord = lams_accord[np.argmin(accord_sp2_ep_bics_02)]
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
# set w_ij = w_ji = 0 if at least one of them is 0 and choose the smaller nonzero otherwise
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
nonzero_indices = np.where((Omega_hat != 0) & (Omega_hat.T != 0))
for i, j in zip(*nonzero_indices):
    if abs(Omega_hat[i, j]) > abs(Omega_hat[j, i]):
        Omega_hat[i, j] = Omega_hat[j, i]
    else:
        Omega_hat[j, i] = Omega_hat[i, j]
Theta_hat = Omega_hat
Rho_hat = partial_corr(Theta_hat)

# TP_norm_accord_sp2_epbic, FP_norm_accord_sp2_epbic, total_norm_accord_sp2_epbic, TP_average_norm_accord_sp2_epbic, FP_average_norm_accord_sp2_epbic = compute_norm(Theta, Theta_hat)
TP_norm_accord_sp2_epbic, FP_norm_accord_sp2_epbic, total_norm_accord_sp2_epbic, TP_average_norm_accord_sp2_epbic, FP_average_norm_accord_sp2_epbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_accord_sp2_epbic = np.linalg.norm(Theta_hat - Theta)

### Run CONCORD

In [8]:
concord_p_bics, concord_ep_bics_02 = [], []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_concord = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_concord = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_concord:
    # CONCORD
    model = GraphicalConcord(Omega_star=np.eye(p), lam1=lam, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Theta_hat = model.omega_.toarray()
    
    concord_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    concord_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_concord = lams_concord[np.argmin(concord_p_bics)]
model = GraphicalConcord(Omega_star=np.eye(p), lam1=best_lam_concord, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Theta_hat = model.omega_.toarray()
Rho_hat = partial_corr(Theta_hat)

# TP_norm_concord_pbic, FP_norm_concord_pbic, total_norm_concord_pbic, TP_average_norm_concord_pbic, FP_average_norm_concord_pbic = compute_norm(Theta, Theta_hat)
TP_norm_concord_pbic, FP_norm_concord_pbic, total_norm_concord_pbic, TP_average_norm_concord_pbic, FP_average_norm_concord_pbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_concord_pbic = np.linalg.norm(Theta_hat - Theta)

# use optimal lambda based on ep-bic
best_lam_concord = lams_concord[np.argmin(concord_ep_bics_02)]
model = GraphicalConcord(Omega_star=np.eye(p), lam1=best_lam_concord, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Theta_hat = model.omega_.toarray()
Rho_hat = partial_corr(Theta_hat)

# TP_norm_concord_epbic, FP_norm_concord_epbic, total_norm_concord_epbic, TP_average_norm_concord_epbic, FP_average_norm_concord_epbic = compute_norm(Theta, Theta_hat)
TP_norm_concord_epbic, FP_norm_concord_epbic, total_norm_concord_epbic, TP_average_norm_concord_epbic, FP_average_norm_concord_epbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_concord_epbic = np.linalg.norm(Theta_hat - Theta)

### Run Glasso

In [9]:
%%capture

glasso_g_bics = []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_glasso = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_glasso = lams[::-1][12:24] # pre-selected lambdas for erdos renyi (diag=[1, 3])

random_state = 2023
np.random.seed(random_state)
for lam in lams_glasso:
    quic = QuicGraphicalLasso(lam=lam, max_iter=100, init_method='cov', auto_scale=False).fit(X)
    Theta_hat = quic.precision_

    glasso_g_bics.append(gauss_BIC(X, Theta_hat))

# use optimal lambda based on g-bic
best_lam_glasso = lams_glasso[np.argmin(glasso_g_bics)]
quic = QuicGraphicalLasso(lam=best_lam_glasso, max_iter=100, init_method='cov', auto_scale=False).fit(X)
Theta_hat = quic.precision_
Rho_hat = partial_corr(Theta_hat)

# TP_norm_glasso, FP_norm_glasso, total_norm_glasso, TP_average_norm_glasso, FP_average_norm_glasso = compute_norm(Theta, Theta_hat)
TP_norm_glasso, FP_norm_glasso, total_norm_glasso, TP_average_norm_glasso, FP_average_norm_glasso = compute_norm(Rho, Rho_hat)
# theta_fro_norm_glasso = np.linalg.norm(Theta_hat - Theta)

### Run SPACE

In [10]:
%%capture

# import SPACE
rpy2.robjects.numpy2ri.activate()
space = importr('space')

space_p_bics, space_ep_bics_02 = [], []

if graph_structure == 'hub_network':
    lams_space = np.logspace(np.log10(40), np.log10(200), 12)
elif graph_structure == 'erdos_renyi':
    lams_space = np.logspace(np.log10(40), np.log10(200), 12) # pre-selected lambdas for erdos renyi (diag=[1,3])

random_state = 2023
np.random.seed(random_state)
for lam in lams_space:
    prec = space.space_joint(X, np.array([lam]))
    Theta_hat = np.array(prec[0])

    space_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    space_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_space = lams_space[np.argmin(space_p_bics)]
prec = space.space_joint(X, np.array([best_lam_space]))
Theta_hat = np.array(prec[0])
Rho_hat = partial_corr(Theta_hat)

# TP_norm_space_pbic, FP_norm_space_pbic, total_norm_space_pbic, TP_average_norm_space_pbic, FP_average_norm_space_pbic = compute_norm(Theta, Theta_hat)
TP_norm_space_pbic, FP_norm_space_pbic, total_norm_space_pbic, TP_average_norm_space_pbic, FP_average_norm_space_pbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_space_pbic = np.linalg.norm(Theta_hat - Theta)

# use optimal lambda based on ep-bic
best_lam_space = lams_space[np.argmin(space_ep_bics_02)]
prec = space.space_joint(X, np.array([best_lam_space]))
Theta_hat = np.array(prec[0])
Rho_hat = partial_corr(Theta_hat)

# TP_norm_space_epbic, FP_norm_space_epbic, total_norm_space_epbic, TP_average_norm_space_epbic, FP_average_norm_space_epbic = compute_norm(Theta, Theta_hat)
TP_norm_space_epbic, FP_norm_space_epbic, total_norm_space_epbic, TP_average_norm_space_epbic, FP_average_norm_space_epbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_space_epbic = np.linalg.norm(Theta_hat - Theta)

In [11]:
print('ACCORD')
print('Total norm (based on p-bic):', total_norm_accord_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_accord_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_epbic, '\n')
print('ACCORD (sparsified & averaged)')
print('Total norm (based on p-bic):', total_norm_accord_sp1_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_sp1_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_sp1_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_accord_sp1_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_sp1_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_sp1_epbic, '\n')
print('ACCORD (sparsified & smaller nonzeros taken)')
print('Total norm (based on p-bic):', total_norm_accord_sp2_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_sp2_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_sp2_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_accord_sp2_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_sp2_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_sp2_epbic, '\n')
print('CONCORD')
print('Total norm (based on p-bic):', total_norm_concord_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_concord_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_concord_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_concord_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_concord_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_concord_epbic, '\n')
print('Glasso')
print('Total norm (based on g-bic):', total_norm_glasso)
print('TP average norm (based on g-bic):', TP_average_norm_glasso)
print('FP average norm (based on g-bic):', FP_average_norm_glasso, '\n')
print('SPACE')
print('Total norm (based on p-bic):', total_norm_space_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_space_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_space_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_space_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_space_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_space_epbic, '\n')

ACCORD
Total norm (based on p-bic): 13.363706122583743
TP average norm (based on p-bic): 0.0060289574004118145
FP average norm (based on p-bic): 0.00018495443625958264 

Total norm (based on ep-bic): 14.130221305990837
TP average norm (based on ep-bic): 0.007477302741866719
FP average norm (based on ep-bic): 0.0002874202688149755 

ACCORD (sparsified & averaged)
Total norm (based on p-bic): 11.011790104673452
TP average norm (based on p-bic): 0.004411027946017372
FP average norm (based on p-bic): 0.0005287856547004569 

Total norm (based on ep-bic): 11.011790104673452
TP average norm (based on ep-bic): 0.004411027946017372
FP average norm (based on ep-bic): 0.0005287856547004569 

ACCORD (sparsified & smaller nonzeros taken)
Total norm (based on p-bic): 11.621618307502946
TP average norm (based on p-bic): 0.004819782484665948
FP average norm (based on p-bic): 0.0004034186913950843 

Total norm (based on ep-bic): 12.107010331859318
TP average norm (based on ep-bic): 0.005276745098745504

In [12]:
print('ACCORD')
print('Total norm (based on p-bic):', total_norm_accord_pbic)
print('TP norm (based on p-bic):', TP_norm_accord_pbic)
print('FP norm (based on p-bic):', FP_norm_accord_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_accord_epbic)
print('TP norm (based on ep-bic):', TP_norm_accord_epbic)
print('FP norm (based on ep-bic):', FP_norm_accord_epbic, '\n')
print('ACCORD (sparsified & averaged)')
print('Total norm (based on p-bic):', total_norm_accord_sp1_pbic)
print('TP norm (based on p-bic):', TP_norm_accord_sp1_pbic)
print('FP norm (based on p-bic):', FP_norm_accord_sp1_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_accord_sp1_epbic)
print('TP norm (based on ep-bic):', TP_norm_accord_sp1_epbic)
print('FP norm (based on ep-bic):', FP_norm_accord_sp1_epbic, '\n')
print('ACCORD (sparsified & smaller nonzeros taken)')
print('Total norm (based on p-bic):', total_norm_accord_sp2_pbic)
print('TP norm (based on p-bic):', TP_norm_accord_sp2_pbic)
print('FP norm (based on p-bic):', FP_norm_accord_sp2_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_accord_sp2_epbic)
print('TP norm (based on ep-bic):', TP_norm_accord_sp2_epbic)
print('FP norm (based on ep-bic):', FP_norm_accord_sp2_epbic, '\n')
print('CONCORD')
print('Total norm (based on p-bic):', total_norm_concord_pbic)
print('TP norm (based on p-bic):', TP_norm_concord_pbic)
print('FP norm (based on p-bic):', FP_norm_concord_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_concord_epbic)
print('TP norm (based on ep-bic):', TP_norm_concord_epbic)
print('FP norm (based on ep-bic):', FP_norm_concord_epbic, '\n')
print('Glasso')
print('Total norm (based on g-bic):', total_norm_glasso)
print('TP norm (based on g-bic):', TP_norm_glasso)
print('FP norm (based on g-bic):', FP_norm_glasso, '\n')
print('SPACE')
print('Total norm (based on p-bic):', total_norm_space_pbic)
print('TP norm (based on p-bic):', TP_norm_space_pbic)
print('FP norm (based on p-bic):', FP_norm_space_pbic, '\n')
print('Total norm (based on ep-bic):', total_norm_space_epbic)
print('TP norm (based on ep-bic):', TP_norm_space_epbic)
print('FP norm (based on ep-bic):', FP_norm_space_epbic, '\n')

ACCORD
Total norm (based on p-bic): 13.363706122583743
TP norm (based on p-bic): 10.424067345312027
FP norm (based on p-bic): 0.6201522247783806 

Total norm (based on ep-bic): 14.130221305990837
TP norm (based on ep-bic): 10.101836004261937
FP norm (based on ep-bic): 0.329383628061962 

ACCORD (sparsified & averaged)
Total norm (based on p-bic): 11.011790104673452
TP norm (based on p-bic): 8.663258885978118
FP norm (based on p-bic): 1.3410004203203585 

Total norm (based on ep-bic): 11.011790104673452
TP norm (based on ep-bic): 8.663258885978118
FP norm (based on ep-bic): 1.3410004203203585 

ACCORD (sparsified & smaller nonzeros taken)
Total norm (based on p-bic): 11.621618307502946
TP norm (based on p-bic): 9.466052799883922
FP norm (based on p-bic): 1.0230698013779338 

Total norm (based on ep-bic): 12.107010331859318
TP norm (based on ep-bic): 9.582569099321836
FP norm (based on ep-bic): 0.829028166245073 

CONCORD
Total norm (based on p-bic): 12.085359596337202
TP norm (based on 

In [ ]:
print('ACCORD')
print('fro. norm (based on p-bic):', theta_fro_norm_accord_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_accord_epbic, '\n')
print('ACCORD (sparsified & averaged)')
print('fro. norm (based on p-bic):', theta_fro_norm_accord_sp1_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_accord_sp1_epbic, '\n')
print('ACCORD (sparsified & smaller nonzeros taken)')
print('fro. norm (based on p-bic):', theta_fro_norm_accord_sp2_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_accord_sp2_epbic, '\n')
print('CONCORD')
print('fro. norm (based on p-bic):', theta_fro_norm_concord_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_concord_epbic, '\n')
print('Glasso')
print('fro. norm (based on g-bic):', theta_fro_norm_glasso, '\n')
print('SPACE')
print('fro. norm (based on p-bic):', theta_fro_norm_space_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_space_epbic)

ACCORD
fro. norm (based on p-bic): 110.75042643770229
fro. norm (based on ep-bic): 115.94371695360532 

ACCORD (sparsified & averaged)
fro. norm (based on p-bic): 93.0722411431842
fro. norm (based on ep-bic): 93.0722411431842 

ACCORD (sparsified & smaller nonzeros taken)
fro. norm (based on p-bic): 135.41178541900123
fro. norm (based on ep-bic): 137.02514991667343 

CONCORD
fro. norm (based on p-bic): 138.23075327173183
fro. norm (based on ep-bic): 140.94117424121217 

Glasso
fro. norm (based on g-bic): 111.43252765076926 

SPACE
fro. norm (based on p-bic): 162.82916085057226
fro. norm (based on ep-bic): 162.82916085057226


In [ ]:
print('ACCORD')
print('fro. norm (based on p-bic):', theta_fro_norm_accord_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_accord_epbic, '\n')
print('ACCORD (sparsified & averaged)')
print('fro. norm (based on p-bic):', theta_fro_norm_accord_sp1_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_accord_sp1_epbic, '\n')
print('ACCORD (sparsified & smaller nonzeros taken)')
print('fro. norm (based on p-bic):', theta_fro_norm_accord_sp2_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_accord_sp2_epbic, '\n')
print('CONCORD')
print('fro. norm (based on p-bic):', theta_fro_norm_concord_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_concord_epbic, '\n')
print('Glasso')
print('fro. norm (based on g-bic):', theta_fro_norm_glasso, '\n')
print('SPACE')
print('fro. norm (based on p-bic):', theta_fro_norm_space_pbic)
print('fro. norm (based on ep-bic):', theta_fro_norm_space_epbic)

ACCORD
fro. norm (based on p-bic): 79.41741959502018
fro. norm (based on ep-bic): 91.16337875422968 

ACCORD (sparsified & averaged)
fro. norm (based on p-bic): 72.65649506318617
fro. norm (based on ep-bic): 72.65649506318617 

ACCORD (sparsified & smaller nonzeros taken)
fro. norm (based on p-bic): 118.6561375217403
fro. norm (based on ep-bic): 118.6561375217403 

CONCORD
fro. norm (based on p-bic): 118.38767830540559
fro. norm (based on ep-bic): 121.99870385861786 

Glasso
fro. norm (based on g-bic): 77.12205117393322 

SPACE
fro. norm (based on p-bic): 143.94683079231103
fro. norm (based on ep-bic): 143.94683079231103


### Weird?

In [ ]:
# true precision values
w_ii = 7.79725419
w_jj = 6.70141228
w_ij = 0.64845515
w_ji = 0.64845515

# estimated precision values (much better diagonal & slightly worse off-diagonal)
w_ii_hat1 = 6.5
w_jj_hat1 = 5.5
w_ij_hat1 = 0.06
w_ji_hat1 = 0.06

# estimated precision values (much worse diagonal & slightly better off-diagonal)
w_ii_hat2 = 4
w_jj_hat2 = 2
w_ij_hat2 = 0.07
w_ji_hat2 = 0.07

In [ ]:
rho_ii = 1
rho_jj = 1
rho_ij = -w_ij/(np.sqrt(w_ii*w_jj))
rho_ji = -w_ji/(np.sqrt(w_jj*w_ii))

rho_ii_hat1 = 1
rho_jj_hat1 = 1
rho_ij_hat1 = -w_ij_hat1/(np.sqrt(w_ii_hat1*w_jj_hat1))
rho_ji_hat1 = -w_ji_hat1/(np.sqrt(w_jj_hat1*w_ii_hat1))

rho_ii_hat2 = 1
rho_jj_hat2 = 1
rho_ij_hat2 = -w_ij_hat2/(np.sqrt(w_ii_hat2*w_jj_hat2))
rho_ji_hat2 = -w_ji_hat2/(np.sqrt(w_jj_hat2*w_ii_hat2))

In [ ]:
print('Frobenius norm of precision matrices', '\n')
print(np.sqrt((w_ii - w_ii_hat1)**2 + (w_jj - w_jj_hat1)**2 + (w_ij - w_ij_hat1)**2 + (w_ji - w_ji_hat1)**2))
print(np.sqrt((w_ii - w_ii_hat2)**2 + (w_jj - w_jj_hat2)**2 + (w_ij - w_ij_hat2)**2 + (w_ji - w_ji_hat2)**2))
print(' ')
print('Frobenius norm of partial correlation matrices', '\n')
print(np.sqrt((rho_ii - rho_ii_hat1)**2 + (rho_jj - rho_jj_hat1)**2 + (rho_ij - rho_ij_hat1)**2 + (rho_ji - rho_ji_hat1)**2))
print(np.sqrt((rho_ii - rho_ii_hat2)**2 + (rho_jj - rho_jj_hat2)**2 + (rho_ij - rho_ij_hat2)**2 + (rho_ji - rho_ji_hat2)**2))

Frobenius norm of precision matrices 

1.9541798349001562
6.098494693867528
 
Frobenius norm of partial correlation matrices 

0.11267324046572862
0.0918647379694666
